# 第18章 with、match和else块

## 18.1 上下文管理器和with块

### 18.1.1 with语句

- with语句作用：简化一些常用的try/finally结构，这种结构保证一段代码运行完毕后执行某些操作，finally子句中的代码通常用于释放重要的资源，或者还原临时改变的状态。

In [7]:
import sys

class LookingGlass:

    def __enter__(self):
        self.original_write = sys.stdout.write
        # 打上猴子补丁
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'

    def reverse_write(self, text):
        # 反转参数的内容
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        # 将原来的方法还原
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True

In [8]:
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


上述打印的结果是反向的。

In [9]:
what

'JABBERWOCKY'

In [10]:
print('Back to normal.')

Back to normal.


### 18.1.2 @contextmanager装饰器

@contextmanager装饰器把简单的生成器函数变成上下文管理器，避免创建类去实现上下文管理器协议。

In [11]:
import contextlib
import sys

@contextlib.contextmanager
def looking_glass():
    # 保留原来的方法
    original_write = sys.stdout.write  

    def reverse_write(text):
        # 反转参数
        original_write(text[::-1])
    
    # 打上猴子补丁
    sys.stdout.write = reverse_write  
    yield 'JABBERWOCKY'
    # 还原方法
    sys.stdout.write = original_write

with块终止时，`__exit__`方法执行了：
1. 检查有没有把异常传给`exc_type`，如果有，则调用`gen.throw(exception)`，再生成器函数主体中yield关键字所在的行抛出异常。
2. 否则，调用`next(gen)`，恢复执行生成器函数主体中yield后面的代码。

@contextmanager装饰的生成器也可以做装饰器。

In [13]:
@looking_glass()
def verse():
    print('The time has come')

In [14]:
verse()

emoc sah emit ehT


## 18.2 案例分析：lis.py中的模式匹配

需求：完成Scheme的解释器

### 18.2.1 类型定义

In [15]:
import math
import operator as op
from collections import ChainMap
from itertools import chain
from typing import Any, TypeAlias, NoReturn

# str的别名，表示标识符
Symbol: TypeAlias = str
# 一种简单的句法元素，表示一个数值或一个Symbol
Atom: TypeAlias = float | int | Symbol
# Scheme程序的基本单元，由原子结构和列表（可以嵌套）构成的表达式    
Expression: TypeAlias = Atom | list

### 18.2.2 解释器

In [16]:
def parse(program: str) -> Expression:
    "从字符串中读取Scheme表达式"
    return read_from_tokens(tokenize(program))

def tokenize(s: str) -> list[str]:
    "把字符串转换成词法单元列表"
    return s.replace('(', ' ( ').replace(')', ' ) ').split()

def read_from_tokens(tokens: list[str]) -> Expression:
    "从一系列词法单元中读取表达式"
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF while reading')
    token = tokens.pop(0)
    if '(' == token:
        exp = []
        while tokens[0] != ')':
            exp.append(read_from_tokens(tokens))
        tokens.pop(0)  # discard ')'
        return exp
    elif ')' == token:
        raise SyntaxError('unexpected )')
    else:
        return parse_atom(token)

def parse_atom(token: str) -> Atom:
    "将数值转成数值类型，将其他符号转成Symbol类型"
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return Symbol(token)

In [17]:
parse('(gcd 18 45)')

['gcd', 18, 45]

In [18]:
parse('''
(define double
    (lambda (n)
        (* n 2)))
''')

['define', 'double', ['lambda', ['n'], ['*', 'n', 2]]]

### 18.2.3 环境

1. Environment类，可以把多个字典和其他映射结合起来，使它们在逻辑上显示并表现为一个整体，并支持就地更改项。

In [19]:
class Environment(ChainMap[Symbol, Any]):
    "ChainMap的子类，允许就地更改项"

    def change(self, key: Symbol, value: Any) -> None:
        "找到key在何处定义，更新对应的值"
        for map in self.maps:
            if key in map:
                map[key] = value  # type: ignore[index]
                return
        raise KeyError(key)

In [20]:
inner_env = {'a': 2}
outer_env = {'a': 0, 'b': 1}
env = Environment(inner_env, outer_env)

In [21]:
env['a']

2

In [22]:
env.change('b', 333)

In [23]:
env

Environment({'a': 2}, {'a': 0, 'b': 333})

2. standard_env函数，构建Environment，并设置Scheme运算逻辑。

In [24]:
def standard_env() -> Environment:
    "An environment with some Scheme standard procedures."
    env = Environment()
    env.update(vars(math))   # sin, cos, sqrt, pi, ...
    env.update({
            '+': op.add,
            '-': op.sub,
            '*': op.mul,
            '/': op.truediv,
            'quotient': op.floordiv,
            '>': op.gt,
            '<': op.lt,
            '>=': op.ge,
            '<=': op.le,
            '=': op.eq,
            'abs': abs,
            'append': lambda *args: list(chain(*args)),
            'apply': lambda proc, args: proc(*args),
            'begin': lambda *x: x[-1],
            'car': lambda x: x[0],
            'cdr': lambda x: x[1:],
            'cons': lambda x, y: [x] + y,
            'display': lambda x: print(lispstr(x)),
            'eq?': op.is_,
            'equal?': op.eq,
            'filter': lambda *args: list(filter(*args)),
            'length': len,
            'list': lambda *x: list(x),
            'list?': lambda x: isinstance(x, list),
            'map': lambda *args: list(map(*args)),
            'max': max,
            'min': min,
            'not': op.not_,
            'null?': lambda x: x == [],
            'number?': lambda x: isinstance(x, (int, float)),
            'procedure?': callable,
            'round': round,
            'symbol?': lambda x: isinstance(x, Symbol),
    })
    return env

### 18.2.4 REPL

In [26]:
def repl(prompt: str = 'lis.py> ') -> NoReturn:
    "一个提示-读取-求值-输出的循环"
    global_env = Environment({}, standard_env())
    while True:
        # 解析语法树
        ast = parse(input(prompt))
        # 计算结果
        val = evaluate(ast, global_env)
        if val is not None:
            # 展示结果
            print(lispstr(val))

def lispstr(exp: object) -> str:
    "将Python对象转换成Lisp理解的字符串"
    if isinstance(exp, list):
        return '(' + ' '.join(map(lispstr, exp)) + ')'
    else:
        return str(exp)

### 18.2.5 求值函数

In [28]:
KEYWORDS = ['quote', 'if', 'lambda', 'define', 'set!']

def evaluate(exp: Expression, env: Environment) -> Any:
    "在环境中求解表达式"
    match exp:
        case int(x) | float(x):
            # 求解数值
            return x
        case Symbol(var):
            # 求解符号
            return env[var]
        case ['quote', x]:
            # (quoto ...)表达式
            return x
        case ['if', test, consequence, alternative]:
            # if表达式
            if evaluate(test, env):
                return evaluate(consequence, env)
            else:
                return evaluate(alternative, env)
        case ['lambda', [*parms], *body] if body:
            # lambda表达式
            return Procedure(parms, body, env)
        case ['define', Symbol(name), value_exp]:
            # 将value_exp表达式的计算结果定义为name
            env[name] = evaluate(value_exp, env)
        case ['define', [Symbol(name), *parms], *body] if body:
            # 自定义运算表达式
            env[name] = Procedure(parms, body, env)
        case ['set!', Symbol(name), value_exp]:
            # 更改已定义变量的值
            env.change(name, evaluate(value_exp, env))
        case [func_exp, *args] if func_exp not in KEYWORDS:
            # 函数调用
            proc = evaluate(func_exp, env)
            values = [evaluate(arg, env) for arg in args]
            return proc(*values)
        case _:
            raise SyntaxError(lispstr(exp))

In [29]:
evaluate(parse('1.5'), {})

1.5

In [30]:
evaluate(parse('+'), standard_env())

<function _operator.add(a, b, /)>

In [31]:
evaluate(parse('(quote no-such-name)'), standard_env())

'no-such-name'

In [32]:
evaluate(parse('(if (= 3 3) 1 0)'), standard_env())

1

### 18.2.6 实现闭包的Procedure类

In [34]:
class Procedure:
    "用户定义的Scheme过程"

    def __init__(self, parms: list[Symbol], body: list[Expression], env: Environment):
        # 函数名称
        self.parms = parms
        # 主体表达式
        self.body = body
        # 环境
        self.env = env

    def __call__(self, *args: Expression) -> Any:
        # 构建函数名称与参数的映射
        local_env = dict(zip(self.parms, args))
        # 构建新的环境
        env = Environment(local_env, self.env)
        # 迭代表达式，在新环境中求解
        for exp in self.body:
            result = evaluate(exp, env)
        return result

In [35]:
expr = '(lambda (a b) (* (/ a b) 100))'
f = evaluate(parse(expr), standard_env())
f(15, 20)

75.0

## 18.3 else语句

- for-else：仅当for循环运行完毕时，才运行else块。
- while-else：仅当while循环因条件为假值而退出时，才运行else块。
- try-else：仅当try块没有抛出异常时，才运行else块。

- EAFP：取得原谅比获得许可容易（Easier to ask for forgiveness than permission）。先假定存在有效的键或属性，如果假设不成立，那么捕获异常。
- LBYL：三思而后行（Look before you leap）。在调用函数或查找属性或键之前，显式测试前提条件。

## 18.4 杂谈

- 真尾调用：通过递归迭代替代命令式语言中的while循环。
- 尾调用：一个函数返回一个函数（可以是同一个函数，也可以不是）调用的结果。
- 对于支持真尾调用的语言，解释器遇到尾调用时，会跳入被调用函数的主体，不创建新的栈帧，可以节省内存。
- Guido van Rossum和其他人的评论：真尾调用提高了所有人的调试难度，只有少数坚持使用递归进行迭代的人能从中受益。